In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fct_invoice.csv')
df

,invoice_no,customer_id,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,229,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,3433,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,644,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,4900,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,1089,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...
99452,I219422,2344,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,172,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,425,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,577,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


## EASY

In [3]:
#1 - count the number of unique customers:

print(str(df['customer_id'].nunique())+' customers')

5191 customers


In [4]:
#2 - product categories + number of product types

print(df['category'].unique())

print(' ') #just a spacing because i'm OC

print(str(df['category'].nunique())+' product categories')

['Clothing' 'Shoes' 'Books' 'Cosmetics' 'Food & Beverage' 'Toys'
 'Technology' 'Souvenir']
 
8 product categories


In [5]:
#3 - most common payment method, how many times was it used

print(df['payment_method'].unique()) #first identify what payment methods were used

CreditCard = df['payment_method'].value_counts()['Credit Card']
DebitCard = df['payment_method'].value_counts()['Debit Card']
Cash = df['payment_method'].value_counts()['Cash']
paylist = [CreditCard, DebitCard, Cash] #creates a list of each MoP's usage numbers

print(' ')

print(str(df['payment_method'].value_counts().idxmax()) #searches for most frequent value in column
      +' was the most common payment method')

print(' ')

print(str(paylist[paylist.index(max(CreditCard, DebitCard, Cash))])+' times used') #prints the max value in list

['Credit Card' 'Debit Card' 'Cash']
 
Cash was the most common payment method
 
44447 times used


## MEDIUM

In [6]:
#1 - top 3 most popular categories, by total sales

catsales = df.copy()
catsales['total sales'] = catsales['price'] *catsales['quantity']

catsales[['category','total sales']].groupby(['category']).agg({'total sales':'sum'}).sort_values(['total sales'], ascending = False).head(3)

#display only category, quantity, and price
#first group all values by the product category
#then sum all the quantities of sales by product category
#sort by price and make it descending order



,total sales
category,
Clothing,1.139968e+08
Shoes,6.655345e+07
Technology,5.786235e+07


In [7]:
jsondf = pd.read_json(r"dim_customer.json")

jsondf.sort_values(['id'])

#sort the dataframe by customer IDs, just in case it isn't yet

,age,first_name,gender,id,last_name
0,48.0,Nicholas,M,0,Flores
1,NaN,Jeffery,M,1,Rowe
2,57.0,Alexis,F,2,Benton
3,73.0,Amy,F,3,Johnson
4,20.0,David,M,4,Moore
...,...,...,...,...,...
5186,NaN,Arthur,M,9428,Harris
5187,NaN,David,M,9494,Roy
5188,85.0,Thomas,M,10350,Turner
5189,NaN,April,F,10587,Sanchez


In [8]:
df['total_sales'] = df['quantity']*df['price']
df.groupby(['customer_id']).agg({'total_sales':'sum'}).sort_values(['customer_id'])
#group the invoice dataframe by combining similar customer IDs
#sum their purchase quantities
#sort by customer ID

,total_sales
customer_id,
0,294676.66
1,205416.23
2,239494.43
3,169732.63
4,308665.21
...,...
9428,1016.50
9494,40.66
10350,162.64


In [9]:
#2 - total sales from customers over age 45

#the two dataframes above are preemptive work to show how the data was sorted and grouped

new_invoice_df = df.groupby(['customer_id']).agg({'total_sales':'sum'}).sort_values(['customer_id']).copy()
new_customer_df = jsondf.sort_values(['id']).copy()

over45_df = new_invoice_df.join(new_customer_df['age']).copy()

final_df = over45_df.loc[over45_df['age'] > 45].copy()

a = final_df.total_sales.sum()

print(str("{:.2f}".format(a)) +' pesos in total sales from customers over 45')

#create a new df to join the 'age' column to the new invoice df
#creater another final df because the over_45 df wont work for some reason
#sum the quantity and print

82053156.99 pesos in total sales from customers over 45


In [10]:
#3 - seasonal trends based on dates, create graph

%matplotlib inline

newdf = pd.read_csv('fct_invoice.csv')
newdf['invoice_date'] = pd.to_datetime(newdf['invoice_date'], format='%d/%m/%Y')
newdf['invoice_month'] = newdf['invoice_date'].dt.month
newdf['invoice_year'] = newdf['invoice_date'].dt.year                                                        

#create a new df because the old df wont work for some reason
#create a new column for month, and another column for year
#ignore the day of date

newdf.sort_values(['invoice_year','invoice_month'])

experidf = newdf.sort_values(['invoice_year','invoice_month']).copy()

grouped_data = experidf.groupby(['invoice_year', 'invoice_month']).agg({'quantity': 'sum'}).reset_index()
grouped_data.sort_values(['invoice_year', 'invoice_month'], inplace=True)

# Create a line graph with different lines for each year
fig, ax = plt.subplots(figsize=(10, 6))

# Iterate over each year in the grouped_data DataFrame
for year in grouped_data['invoice_year'].unique():
    year_data = grouped_data[grouped_data['invoice_year'] == year]
    
    # Plot a line for each year
    ax.plot(year_data['invoice_month'], year_data['quantity'], label=year, marker='o')

# Set the x-axis and y-axis labels
ax.set_xlabel('Month')
ax.set_ylabel('Quantity')

# Set the title of the graph
ax.set_title('Quantity Bought per Month (by Year)')

# Display a legend
ax.legend()

# Show the plot
plt.show()

#made with the assistance of ChatGPT

NameError: name 'plt' is not defined

The graph above shows the total products purchased per month, per year. In 2021, the quantity of products purchased was at its highest during July after a noticeable lower quantity of sales in previous months. 

In 2022, March had the highest number of sales after a poor sales months in February. The amount of sales then stabilizes for the rest of the year. 

As for 2023, the data is not sufficient enough to determine any trend yet.

Regardless, for the years 2021 and 2022, there seems to be a small, yet noticeable trend of higher sales towards the second half of the year. This could be due to the approaching holidays, so gift purchasing would be more frequent.

# HARD

In [20]:
#1 - pivot table showing breakdown of sales across ff. dimensions:
#in this order: category, decade age range (e.g., 10-19, 20-29, and so forth)

import pandas as pd
import numpy as np

df1 = pd.read_csv('fct_invoice.csv')
df2 = pd.read_json(r"dim_customer.json")

df1['total_sales'] = df1['price'] * df1['quantity']
df1_clean = df1[['customer_id', 'total_sales', 'category']].copy()

df1_clean['age'] = df1_clean['customer_id'].map(df2.set_index('id')['age'])

df1_clean_sorted = df1_clean.sort_values(['category', 'age'])

age_ranges = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

age_labels = [f"{age}-{age + 9}" for age in age_ranges[:-1]]

df1_clean_sorted['age_range'] = pd.cut(df1_clean_sorted['age'], bins=age_ranges, labels=age_labels, right=False)

df_grouped = df1_clean_sorted.groupby(['category', 'age_range']).agg({'total_sales': 'sum'}).reset_index()

pivot = df_grouped.pivot_table(index=['category', 'age_range'], values=['total_sales'], aggfunc='sum')

pd.set_option('display.max_rows', None)

pivot

#made with the assistance of ChatGPT

total_sales
category        age_range             
Books           10-19         12801.75
                20-29         79461.75
                30-39         84143.10
                40-49         87203.40
                50-59         74962.20
                60-69         80476.80
                70-79         54827.85
                80-89         26664.00
                90-99             0.00
Clothing        10-19       1927413.84
                20-29      10250732.80
                30-39      11354126.96
                40-49      11215189.92
                50-59      10321851.76
                60-69      10660642.08
                70-79       7769371.28
                80-89       4493698.00
                90-99             0.00
Cosmetics       10-19        109619.36
                20-29        605915.32
                30-39        723544.70
                40-49        665034.96
                50-59        587780.96
                60-69        649990.76
                70-79        419082.62
                80-89        254531.60
                90-99             0.00
Food & Beverage 10-19         13582.31
                20-29         73491.96
                30-39         87246.86
                40-49         89553.29
                50-59         72425.04
                60-69         80976.09
                70-79         55050.98
                80-89         31332.93
                90-99             0.00
Shoes           10-19        993281.35
                20-29       5924878.24
                30-39       7200839.66
                40-49       7025590.02
                50-59       5852857.84
                60-69       5978893.54
                70-79       4480269.05
                80-89       2628744.60
                90-99             0.00
Souvenir        10-19         12375.15
                20-29         54943.32
                30-39         64092.72
                40-49         66391.80
                50-59         54943.32
                60-69         55025.43
                70-79         38673.81
                80-89         23718.06
                90-99             0.00
Technology      10-19       1129800.00
                20-29       5261550.00
                30-39       6159300.00
                40-49       5954550.00
                50-59       4435200.00
                60-69       5471550.00
                70-79       3638250.00
                80-89       1946700.00
                90-99             0.00
Toys            10-19         60426.24
                20-29        385100.80
                30-39        420582.40
                40-49        390906.88
                50-59        345067.52
                60-69        363955.20
                70-79        273244.16
                80-89        153932.80
                90-99             0.00